[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1SH81IOt4NYOaKVrL5Sa2tt_HeQqduU71?usp=sharing)

# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="400" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝.

In [1]:
!nvidia-smi

Mon Feb 26 06:05:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


# 2) 데이터셋 준비

### 2.1) 데이터셋: KoAlpaca v1.1a
- [ref](https://github.com/Beomi/KoAlpaca#%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-v11-%EC%A0%9C%EC%9E%91-%EB%B0%A9%EB%B2%95)

In [3]:
from datasets import load_dataset
from pprint import pprint

data_koalpaca = load_dataset("beomi/KoAlpaca-v1.1a", split="train")

print(data_koalpaca)

# data 포맷 변환
data_koalpaca = data_koalpaca.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
pprint(data_koalpaca[0]['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21155
})


Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

('### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n'
 '\n'
 '### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n'
 '\n'
 '식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. '
 '고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 '
 '부분입니다.\n'
 '\n'
 ' 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n'
 '\n'
 '고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. '
 '다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>')


### 2.2) 데이터셋: 구름 데이터셋 v2
- [ref](https://github.com/nlpai-lab/KULLM#%EA%B5%AC%EB%A6%84-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-v2)

In [4]:
from datasets import load_dataset

data_kullm_v2 = load_dataset("nlpai-lab/kullm-v2", split="train")
print(data_kullm_v2)

# data 포맷 변환
data_kullm_v2 = data_kullm_v2.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
pprint(data_kullm_v2[0]['text'])

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})


Map:   0%|          | 0/152630 [00:00<?, ? examples/s]

('### 질문: 3원색이란 무엇인가요?\n'
 '\n'
 '### 답변: 세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 '
 '조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, '
 '파랑(RGB)입니다.<|endoftext|>')


### 2.3) 데이터셋: KO-Platypus
- [ref](https://github.com/Marker-Inc-Korea/KO-Platypus?fbclid=IwAR1rV_woLbLZ7zglBYO7RVLI3-5jTykqXxS7sbjWG_Pys8nvAhJB5edtxP0#datasets)

In [5]:
from datasets import load_dataset

data_KO_Platypus = load_dataset("kyujinpy/KOpen-platypus", split="train")
print(data_KO_Platypus)

# data 포맷 변환
data_KO_Platypus = data_KO_Platypus.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)
print(data_KO_Platypus[0]['text'])

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction', 'data_source'],
    num_rows: 24926
})


Map:   0%|          | 0/24926 [00:00<?, ? examples/s]

### 질문: 보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.

### 답변: 모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \frac{1}{3}$, $P(B) = \frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \frac{1}{3} - frac{5}{12} = \frac{12}{12} - frac{4}{12} - frac{5}{12} = \frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \frac{1}{4}$입니다.<|endoftext|>


## 2.4) 데이터셋 선택
1) KoAlpaca v1.1a
2) 구름 데이터셋 v2
3) KO-Platypus
4) 1)+2)+3)

In [6]:
# data = data_kullm_v2
# data = data_koalpaca
# data = data_KO_Platypus

# or 세 데이터셋 병합
from datasets import Dataset
data_list = data_koalpaca['text'] + data_kullm_v2['text'] + data_KO_Platypus['text']
list_dict = []
for tmp in data_list:
    tmp_dict = {}
    tmp_dict['text'] = tmp
    list_dict.append(tmp_dict)
data = Dataset.from_list(list_dict, split="train")

print(data)

Dataset({
    features: ['text'],
    num_rows: 198711
})


## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM에 영향이 적다.

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "EleutherAI/polyglot-ko-12.8b"  # Orininal polyglot
model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

### 텍스트 데이터만 tokenize

In [8]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
print(data[0]['text'])

Map:   0%|          | 0/198711 [00:00<?, ? examples/s]

### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>


PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비.

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,  # Lora scaling, 32, 64, 128
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # SEQ_CLS, SEQ_2_SEQ_LM, TOKEN_CLS
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [12]:
!nvidia-smi

Mon Feb 26 06:18:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              31W /  70W |   8347MiB / 15360MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
!top

=top - 06:18:06 up 15 min,  0 users,  load average: 2.44, 2.44, 1.55
Tasks:  16 total,   1 running,  14 sleeping,   0 stopped,   1 zombie
%Cpu(s):  3.2 us,  6.5 sy,  0.0 ni, 87.1 id,  3.2 wa,  0.0 hi,  0.0 si,  0.0 st
MiB Mem :  12979.0 total,    753.5 free,   3383.4 used,   8842.0 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.   9263.2 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND                      
    783 root      20   0   23.0g   2.6g 336512 S   6.2  20.4   7:26.97 python3                      
      1 root      20   0    1076      8      0 S   0.0   0.0   0:00.06 docker-init                  
      7 root      20   0  894356  56980  30904 S   0.0   0.4   0:08.27 node                         
     17 root      20   0    7376   3340   3040 S   0.0   0.0   0:00.50 oom_monitor.sh               
     19 root      20   0    7376   1912   1628 S   0.0   0.0   0:00.00 run.sh                       
     21 root      20  

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 200개 스텝만 학습해보자.

In [14]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=50, ## 소량만 학습
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 840.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 705.06 MiB is free. Process 8293 has 13.93 GiB memory in use. Of the allocated memory 13.23 GiB is allocated by PyTorch, and 583.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1554: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1354: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1672: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `

tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34,    6,    6,    6,
         4253,   29, 1832,  296, 8678,  270, 1922,  327]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:422: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 우리는 우리가 섭취하는 음식의 질에 달려 있습니다. 우리가 먹는 음식은 우리가 살 수 있는 것뿐만 아니라 우리의 건강에 많은 영향을 미칩니다.좋은 음식에는 유기농 채소, 통곡물, 통유류 및 콩류가 포함됩니다. 더 자세히 살펴보면 ### 건강한 식습관...- ASK.QUEST 웹 사이트를 방문하여 세 가지 건강하게 사는 방법에 대한 더 많은 내용을 확인하세요.### 추가 아이디어...- 유기농 식품을 먹고, 통곡물을 먹고, 콩류를 먹어서 건강한 식습관을 실천하세요.### 출처 https://ask.fm/drdr<|endoftext|>


`### 질문: 건강하게 살기 위한 세 가지 방법은?`

`### 답변: 우리는 우리가 섭취하는 음식의 질에 달려 있습니다. 우리가 먹는 음식은 우리가 살 수 있는 것뿐만 아니라 우리의 건강에 많은 영향을 미칩니다.좋은 음식에는 유기농 채소, 통곡물, 통유류 및 콩류가 포함됩니다. 더 자세히 살펴보면 ### 건강한 식습관...- ASK.QUEST 웹 사이트를 방문하여 세 가지 건강하게 사는 방법에 대한 더 많은 내용을 확인하세요.### 추가 아이디어...- 유기농 식품을 먹고, 통곡물을 먹고, 콩류를 먹어서 건강한 식습관을 실천하세요.### 출처 https://ask.fm/drdr<|endoftext|>`

In [ ]:
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 슈카월드가 무엇인가요?

### 답변: 저스트 두 잇! 슈카월드는 개인 투자자를 위한 유튜브 플랫폼이에요. 많은 개인 투자자가 슈카월드를 통해 투자에 대한 정보와 의견을 얻기 위해 채널을 찾아요. 슈카월드는 단순히 투자 정보를 제공하는 것을 넘어, 유튜버 슈카가 투자에 대한 자신의 생각과 판단을 공유합니다. 이를 통해 개인 투자자로서 무엇을 해야 하고, 무엇을 피해야 하는지 이해하도록 돕죠. 슈카월드의 인기 비결은 투자에 대한 진지한 접근과 그가 공유하는 통찰력 때문입니다. 슈카는 단순히 대중의 관심을 끄는 것만이 아니라 개인 투자자들에게도 투자하는 목적과 방법에 대한 유용한 조언과 영감을 제시해요.#####관련 콘텐츠#####미디어 채널에서 '슈카월드'와 함께 보면 좋은 콘텐츠들:🧐 🇺🇸한국이 아닌 다른 국가에 투자해야 할까요?: 🧐 🇺🇸투자 포트폴리오에 대한 영감을 받고 싶을 때: 🧐 🇺🇸투자의 개념을 더 배우고


`### 질문: 슈카월드가 무엇인가요?`

`### 답변: 저스트 두 잇! 슈카월드는 개인 투자자를 위한 유튜브 플랫폼이에요. 많은 개인 투자자가 슈카월드를 통해 투자에 대한 정보와 의견을 얻기 위해 채널을 찾아요. 슈카월드는 단순히 투자 정보를 제공하는 것을 넘어, 유튜버 슈카가 투자에 대한 자신의 생각과 판단을 공유합니다. 이를 통해 개인 투자자로서 무엇을 해야 하고, 무엇을 피해야 하는지 이해하도록 돕죠. 슈카월드의 인기 비결은 투자에 대한 진지한 접근과 그가 공유하는 통찰력 때문입니다. 슈카는 단순히 대중의 관심을 끄는 것만이 아니라 개인 투자자들에게도 투자하는 목적과 방법에 대한 유용한 조언과 영감을 제시해요.#####관련 콘텐츠#####미디어 채널에서 '슈카월드'와 함께 보면 좋은 콘텐츠들:🧐 🇺🇸한국이 아닌 다른 국가에 투자해야 할까요?: 🧐 🇺🇸투자 포트폴리오에 대한 영감을 받고 싶을 때: 🧐 🇺🇸투자의 개념을 더 배우고`


In [ ]:
gen('공공성을 측정하기위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 공공성을 측정하기위한 방법은?

### 답변:공공성은 한 사회에서 공동의 관심사에 대해 서로 이야기하고, 관심과 생각을 공유하고, 그와 관련된 활동을 통해 행동합니다. 공공에 참여하는 사람들은 공공성을 인식할 수 있습니다.이 질문을 답하기 위해서는 공공성에 대해 설명하는 몇 가지 방법을 살펴봐야 합니다.공공성은 사회적 상호작용 분야에서 인식되는 것이 될 수 있습니다. 이 경우 사회적 상호작용에서 공공성은 어떤 의미를 부여하고 공유하는 생각과 아이디어를 반영하여 인식됩니다. 다른 방법으로는 공공성을 표현할 수있는 매체, 활동을 생각할 수 있습니다. 공공성 여부는 상호작용이 공유 및 인식하는 관심사 또는 문제를 다루는지 여부에 달려있습니다. 예를 들어, 정치적 공공성은 사회 정의, 소득 불평등, 건강 문제, 지구 오염과 같은 이슈를 다루는 문제와 공유된 인식에 좌우됩니다. 따라서 공공성은 관련 정보를 공유하는 모든 사람과 상호 작용합니다.공공성을 측정하는 다른 중요한 방법은 사람들이 공공에 참여하거나 공공적인지 여부를 기준으로 인식, 행동 및 상호작용을 식별하는 것입니다. 여기


`### 질문: 공공성을 측정하기위한 방법은?`

`### 답변: 공공성은 한 사회에서 공동의 관심사에 대해 서로 이야기하고, 관심과 생각을 공유하고, 그와 관련된 활동을 통해 행동합니다. 공공에 참여하는 사람들은 공공성을 인식할 수 있습니다.이 질문을 답하기 위해서는 공공성에 대해 설명하는 몇 가지 방법을 살펴봐야 합니다.공공성은 사회적 상호작용 분야에서 인식되는 것이 될 수 있습니다. 이 경우 사회적 상호작용에서 공공성은 어떤 의미를 부여하고 공유하는 생각과 아이디어를 반영하여 인식됩니다. 다른 방법으로는 공공성을 표현할 수있는 매체, 활동을 생각할 수 있습니다. 공공성 여부는 상호작용이 공유 및 인식하는 관심사 또는 문제를 다루는지 여부에 달려있습니다. 예를 들어, 정치적 공공성은 사회 정의, 소득 불평등, 건강 문제, 지구 오염과 같은 이슈를 다루는 문제와 공유된 인식에 좌우됩니다. 따라서 공공성은 관련 정보를 공유하는 모든 사람과 상호 작용합니다.공공성을 측정하는 다른 중요한 방법은 사람들이 공공에 참여하거나 공공적인지 여부를 기준으로 인식, 행동 및 상호작용을 식별하는 것입니다. 여기`

In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

### 답변: 다음 질문에 나와 있듯이 주식 시장에서 안정적으로수익을 얻는 방법은 많습니다. 여기에는 여러 가지 방법과 다른 전략이 있으며 각 전략은 어떤 유형의투자를위한 가장 좋은 기회를 제공합니다.우선, 우리는 투자자의 투자 목표에 따라 투자를 분류 할 수 있습니다.투자자는 다음 사항을 고려하여 투자 결정을 내릴 수 있습니다.- 장기적으로 투자하려는 투자의 유형- 어떤 수익률 목표를 보유하고자 하는지- 연간 최소 수익률- 연간 최고 수익률- 투자 자금에 대한 연간 최소 수익률- 연간 최대 수익률 등이 있습니다.일반적으로 투자자는 투자의 장기적 가치를 고려하여 투자결정하며 더 긴 기간 동안 투자하면 더 높은 투자 수익률을 제공합니다.일반적으로 투자자가 투자의 단기 가치를 고려하고 투자 결정을 내릴 때투자자는 짧은 투자 기간에 집중적으로 투자함에 따라 더 높은수익률을 달성하려고 시도합니다.따라서 투자자는 투자를 관리하기 위해 더 긴 기간을 고려해야 하며그 결과 투자의 단기 가치를 고려하여 투자 결정을 내리는 것은더 높은 투자 수익률을 얻는 데 도움이 될 수 있습니다.다음으로


`### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?`

`### 답변: 다음 질문에 나와 있듯이 주식 시장에서 안정적으로수익을 얻는 방법은 많습니다. 여기에는 여러 가지 방법과 다른 전략이 있으며 각 전략은 어떤 유형의투자를위한 가장 좋은 기회를 제공합니다.우선, 우리는 투자자의 투자 목표에 따라 투자를 분류 할 수 있습니다.투자자는 다음 사항을 고려하여 투자 결정을 내릴 수 있습니다.- 장기적으로 투자하려는 투자의 유형- 어떤 수익률 목표를 보유하고자 하는지- 연간 최소 수익률- 연간 최고 수익률- 투자 자금에 대한 연간 최소 수익률- 연간 최대 수익률 등이 있습니다.일반적으로 투자자는 투자의 장기적 가치를 고려하여 투자결정하며 더 긴 기간 동안 투자하면 더 높은 투자 수익률을 제공합니다.일반적으로 투자자가 투자의 단기 가치를 고려하고 투자 결정을 내릴 때투자자는 짧은 투자 기간에 집중적으로 투자함에 따라 더 높은수익률을 달성하려고 시도합니다.따라서 투자자는 투자를 관리하기 위해 더 긴 기간을 고려해야 하며그 결과 투자의 단기 가치를 고려하여 투자 결정을 내리는 것은더 높은 투자 수익률을 얻는 데 도움이 될 수 있습니다.다음으로`


In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 옵션 시장에서는 투자자의 포지션에 따라 풋 옵션과 콜 옵션이라는 용어를 사용합니다. 이러한 차이점을 이해하는 것이 매우 중요합니다. 풋 옵션은 투자자에게 현재 시장에서의 가격 하락에 대한 권리를 부여하는 반면, 콜 옵션은 투자자에게 현재 가격 상승에 대한 권리를 부여합니다. 옵션 시장에서 풋 옵션은 일반적으로 높은 프리미엄을 요구하는 반면, 일반적인 콜 옵션은 상대적으로 낮은 프리미엄을 요구합니다. 개인 투자자 또는 전문 투자자와 같은 모든 투자자가 시장에서 풋 옵션을 사용하려면 더 높은 프리미엄을 지불해야 합니다. 개인 투자자 중에는 위험을 감수하는 것이 자신의 평판에 위험 요인이 될 수 있다는 것을 인식하고 풋 옵션을 사용하지 않는 사람들이 종종 있습니다. 그들은 자신을 보호하기 위해 개인 투자 포지션에 콜 옵션을 사용합니다. 전문 투자자들도 마찬가지로 현재 시장 가격에서 잠재적인 수익을 높이는 능력, 즉 콜 옵션을 사용합니다. 풋 옵션은 가격 하락에 대한 권리를 가지고 있고 따라서 풋 옵션 보유자는 가격 하락에 대한 이득을 얻는 것으로 봅니다. 반대로 콜 옵션은 가격 상승에 대한 이득을


`### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: 옵션 시장에서는 투자자의 포지션에 따라 풋 옵션과 콜 옵션이라는 용어를 사용합니다. 이러한 차이점을 이해하는 것이 매우 중요합니다. 풋 옵션은 투자자에게 현재 시장에서의 가격 하락에 대한 권리를 부여하는 반면, 콜 옵션은 투자자에게 현재 가격 상승에 대한 권리를 부여합니다. 옵션 시장에서 풋 옵션은 일반적으로 높은 프리미엄을 요구하는 반면, 일반적인 콜 옵션은 상대적으로 낮은 프리미엄을 요구합니다. 개인 투자자 또는 전문 투자자와 같은 모든 투자자가 시장에서 풋 옵션을 사용하려면 더 높은 프리미엄을 지불해야 합니다. 개인 투자자 중에는 위험을 감수하는 것이 자신의 평판에 위험 요인이 될 수 있다는 것을 인식하고 풋 옵션을 사용하지 않는 사람들이 종종 있습니다. 그들은 자신을 보호하기 위해 개인 투자 포지션에 콜 옵션을 사용합니다. 전문 투자자들도 마찬가지로 현재 시장 가격에서 잠재적인 수익을 높이는 능력, 즉 콜 옵션을 사용합니다. 풋 옵션은 가격 하락에 대한 권리를 가지고 있고 따라서 풋 옵션 보유자는 가격 하락에 대한 이득을 얻는 것으로 봅니다. 반대로 콜 옵션은 가격 상승에 대한 이득을`


In [ ]:
gen('풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변:• 콜옵션이나 풋옵션의 한 가지 포지션은 다른 포지션과 관련된 미래 변동성의 가능한 범위에 달려 있습니다.• 변동성이란 현재 자산, 시장 또는 시장 상황에 영향을 미치는 사건의 기대되는 정도를 나타냅니다.• 자산에 대한 변동성의 기대가 크면 클수록 투자자는 더욱 자주 시장에 참여하게됩니다.• 투자자는 시장 흐름을 자신의 계획에 따라 조정하면서 예상된 목표를 실현하기 위해 시장에 참여합니다.• 이와 같이 거래하는 투자자에게는 시장 참여를 통해 현재 시점에서 기대되는 금액을 실현할 가능성이 커집니다.• 또한, 자산의 전체 잠재력에 접근하는 다른 사람들이 증가함에 따라, 미래에 더 많은 변동성의 더 큰 가능성이 발생하게됩니다.• 콜옵션은 이와 같이 변동성의 기대가 큰 포지션을 대표하며, 특히 많은 변동성의 경우에 발생합니다.• 반대로, 시장이 상승하는 경우에도 콜옵션을 매수하면 미래에 예상되는 금액을 거래할 가능성이 낮아집니다.• 콜옵션에 대한 이러한 포지션을 풋옵션이라고 하며, 특히 많은 변동성의 시장에서


`### 질문: 풋옵션 매도와 콜옵션 매수의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?`

`### 답변: • 콜옵션이나 풋옵션의 한 가지 포지션은 다른 포지션과 관련된 미래 변동성의 가능한 범위에 달려 있습니다.• 변동성이란 현재 자산, 시장 또는 시장 상황에 영향을 미치는 사건의 기대되는 정도를 나타냅니다.• 자산에 대한 변동성의 기대가 크면 클수록 투자자는 더욱 자주 시장에 참여하게됩니다.• 투자자는 시장 흐름을 자신의 계획에 따라 조정하면서 예상된 목표를 실현하기 위해 시장에 참여합니다.• 이와 같이 거래하는 투자자에게는 시장 참여를 통해 현재 시점에서 기대되는 금액을 실현할 가능성이 커집니다.• 또한, 자산의 전체 잠재력에 접근하는 다른 사람들이 증가함에 따라, 미래에 더 많은 변동성의 더 큰 가능성이 발생하게됩니다.• 콜옵션은 이와 같이 변동성의 기대가 큰 포지션을 대표하며, 특히 많은 변동성의 경우에 발생합니다.• 반대로, 시장이 상승하는 경우에도 콜옵션을 매수하면 미래에 예상되는 금액을 거래할 가능성이 낮아집니다.• 콜옵션에 대한 이러한 포지션을 풋옵션이라고 하며, 특히 많은 변동성의 시장에서`


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

### 답변:마진콜은 어떤 조건에 부합할 것을 요구해. 예를 들어, 충분한 담보금을 보유하지 않는 것, 레버리지가 너무 과한 것, 과도한 노출이 있는 것, 잘못된 정보에 근거한 것, 과도하게 많은 돈을 거래에 사용하는 것과 같은 지표.마진콜은 단순히 필요한 행동입니다. 그것은 필요한 조치에요. 마진콜 조건에 해당하면, 거래자는 더 높은 마진을 유지하고 더 높은 레버리지를 적용하기 위해 담보금이 충분한지 확인하거나 자신의 노출을 더 잘 통제해야 합니다.마진콜이 발생하는 주요 이유로는 담보금 부족, 레버리지 과용, 잘못된 정보 탐색, 과도한 많은 돈의 사용과 같이 많으며, 이는 모든 사람들이 알아야 하는 기초 지식입니다.가장 간단한 규칙은 마진콜이 발생할 확률을 높이는 거래와 거래를 수행하지 않는 것입니다. 만약 돈이 당신 앞에 있다면 당신은 거래를 할 수 있습니다. 그러나, 당신이 당신의 돈으로 거래를 시도할 때마다 당신은 이 모든 거래에 마진을 적용해야 합니다. 거래는 항상 리스크가 있기 때문에, 당신


`### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?`

`### 답변: 마진콜은 어떤 조건에 부합할 것을 요구해. 예를 들어, 충분한 담보금을 보유하지 않는 것, 레버리지가 너무 과한 것, 과도한 노출이 있는 것, 잘못된 정보에 근거한 것, 과도하게 많은 돈을 거래에 사용하는 것과 같은 지표.마진콜은 단순히 필요한 행동입니다. 그것은 필요한 조치에요. 마진콜 조건에 해당하면, 거래자는 더 높은 마진을 유지하고 더 높은 레버리지를 적용하기 위해 담보금이 충분한지 확인하거나 자신의 노출을 더 잘 통제해야 합니다.마진콜이 발생하는 주요 이유로는 담보금 부족, 레버리지 과용, 잘못된 정보 탐색, 과도한 많은 돈의 사용과 같이 많으며, 이는 모든 사람들이 알아야 하는 기초 지식입니다.가장 간단한 규칙은 마진콜이 발생할 확률을 높이는 거래와 거래를 수행하지 않는 것입니다. 만약 돈이 당신 앞에 있다면 당신은 거래를 할 수 있습니다. 그러나, 당신이 당신의 돈으로 거래를 시도할 때마다 당신은 이 모든 거래에 마진을 적용해야 합니다. 거래는 항상 리스크가 있기 때문에, 당신`



## 모델 저장 & 업로드

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):

^C


In [ ]:
# model.push_to_hub('beomi/qlora-koalpaca-polyglot-12.8b-50step')

# model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')
model.push_to_hub('kkko/qlora-polyglot-12.8b-50step')

ValueError: ignored

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb

In [ ]:
!pip -q install gradio

In [ ]:
## gradio inference example code
# ref: https://github.com/Beomi/KoAlpaca/blob/main/webui/app.py
import gradio as gr
import torch
from transformers import pipeline, AutoModelForCausalLM

# MODEL = "beomi/KoAlpaca-Polyglot-12.8B"
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     device_map="auto",
#     load_in_8bit=True,
#     revision="8bit",
#     # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
# )

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=MODEL,
#     # device=2,
# )


def answer(state, state_chatbot, text):
    messages = state + [{"role": "질문", "content": text}]

    conversation_history = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )

#     ans = pipe(
#         conversation_history + "\n\n### 답변:",
#         do_sample=True,
#         max_new_tokens=512,
#         temperature=0.7,
#         top_p=0.9,
#         return_full_text=False,
#         eos_token_id=2,
#     )

    gened = model.generate(
        **tokenizer(
#             f"### 질문: {x}\n\n### 답변:",
            f"{conversation_history}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )



#     msg = ans[0]["generated_text"]
    msg = tokenizer.decode(gened[0])


    if "###" in msg:
        msg = msg.split("###")[0]

    new_state = [{"role": "이전 질문", "content": text}, {"role": "이전 답변", "content": msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    print(state)
    print(state_chatbot)

    return state, state_chatbot, state_chatbot


with gr.Blocks(css="#chatbot .overflow-y-auto{height:750px}") as demo:
    state = gr.State(
        [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]
    )
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML(
            """<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatKoAlpaca 12.8B (v1.1b-chat-8bit)</h1>
            </div>
            <div>
                Demo runs on RTX 3090 (24GB) with 8bit quantized
            </div>
        </div>"""
        )

    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Send a message...").style(
            container=False
        )

    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "", None, txt)

demo.launch(debug=True, share=True, server_name="0.0.0.0")